
Las tablas son generadas en **Modo Batch**, y se almacenan como tablas Delta en el Data Lake, siguiendo el modelo Medallón.

## Ejecución

- Todas las tablas se generan de forma **Batch**, utilizando los datos enriquecidos de la capa Silver.
- Los resultados se almacenan como **Tablas Delta** listas para ser consultadas o visualizadas.
- Pueden ser ejecutadas manualmente desde notebooks o programadas, en data factory se opto por la utilizacion de **Schedule Trigger**

---


### 1. Ventas por Día, Canal y Región
- 📌 **Descripción**: Agrega la información de ventas diarias segmentadas por canal y región.
- 📈 **Aplicación**: Monitoreo de rendimiento por zonas y análisis temporal de ventas.


In [0]:
from pyspark.sql.functions import *
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")


sales_by_day = (
    df_gold
    .groupBy("event_date", "channel_name", "store_region")
    .agg(
        count("*").alias("total_transactions"),
        sum("amount").alias("total_sales"),
        avg("amount").alias("avg_ticket")
    )
)

sales_by_day.write.mode("overwrite").format("delta").save("/mnt/gold/sales/sales_by_day")
display(sales_by_day)


### 2. Top Clientes
- 📌 **Descripción**: Identifica los clientes con mayor nivel de gasto total.
- 👥 **Aplicación**: Segmentación de clientes, programas de fidelidad, y campañas de marketing.

In [0]:

df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

top_customers = (
    df_gold
    .groupBy("customer_id", "customer_name", "customer_loyalty_tier")
    .agg(
        sum("amount").alias("total_spent"),
        count("*").alias("transactions"),
        avg("amount").alias("avg_ticket"),
        max("event_date").alias("last_purchase_date")
    )
    .orderBy(desc("total_spent"))
)

display(top_customers)
top_customers.write.mode("overwrite").format("delta").save("/mnt/gold/sales/top_customers")


### 3. Rendimiento de Productos
- 📌 **Descripción**: Evalúa el desempeño de los productos en términos de ventas e ingresos.
- 📦 **Aplicación**: Gestión de inventario, decisiones de pricing y desarrollo de productos.


In [0]:
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

product_performance = (
    df_gold
    .groupBy("product_id", "product_name")
    .agg(
        count("*").alias("units_sold"),
        sum("amount").alias("total_revenue"),
        avg("amount").alias("avg_ticket")
    )
    .orderBy(desc("total_revenue"))
)

display(product_performance)
product_performance.write.mode("overwrite").format("delta").save("/mnt/gold/sales/product_performance")



### 4. Ventas por Región y Canal
- 📌 **Descripción**: Muestra un cruce entre regiones de venta y canales utilizados.
- 🗺️ **Aplicación**: Análisis geográfico y por tipo de canal, útil para visualizaciones tipo mapa de calor o matrices comparativas.

In [0]:
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

sales_by_region_channel = (
    df_gold
    .groupBy("store_region", "channel_name")
    .agg(
        sum("amount").alias("total_sales"),
        count("*").alias("transactions")
    )
)

display(sales_by_region_channel)
sales_by_region_channel.write.mode("overwrite").format("delta").save("/mnt/gold/sales/sales_by_region_channel")


### 5. Distribución Diaria por Categoría de Ticket
- 📌 **Descripción**: Clasifica las ventas diarias según el rango de importe del ticket.
- 🎯 **Aplicación**: Evaluación de hábitos de consumo y detección de tendencias de compra.


In [0]:
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

daily_ticket_distribution = (
    df_gold
    .groupBy("event_date", "ticket_category")
    .agg(
        count("*").alias("transaction_count"),
        sum("amount").alias("total_amount")
    )
    .orderBy("event_date", "ticket_category")
)

display(daily_ticket_distribution)
daily_ticket_distribution.write.mode("overwrite").format("delta").save("/mnt/gold/sales/daily_ticket_distribution")


### 6. Rendimiento por Sucursal (Extra)
- 📌 **Descripción**: Mide el desempeño de cada sucursal en base a sus ventas y frecuencia de transacciones.
- 🏬 **Aplicación**: Comparativa entre locales, decisiones de expansión, análisis por región.


In [0]:
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

store_performance = (
    df_gold
    .groupBy("store", "store_name", "store_region")
    .agg(
        count("*").alias("transactions"),
        sum("amount").alias("total_sales"),
        avg("amount").alias("avg_ticket")
    )
    .orderBy(desc("total_sales"))
)

display(store_performance)
store_performance.write.mode("overwrite").format("delta").save("/mnt/gold/sales/store_performance")


## Creación de tablas en hive metastore

In [0]:
spark.sql(
    """
        CREATE SCHEMA IF NOT EXISTS capa_gold;
    """
)
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.sales_by_day
     USING DELTA
    LOCATION '/mnt/gold/sales/sales_by_day';
    """
)

In [0]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.top_customers
USING DELTA
LOCATION '/mnt/gold/sales/top_customers';
    """
)


In [0]:
spark.sql(
    """
    CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.product_performance
USING DELTA
LOCATION '/mnt/gold/sales/product_performance';
    """
)

In [0]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.sales_by_region_channel
USING DELTA
LOCATION '/mnt/gold/sales/sales_by_region_channel';
    """
)

In [0]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.daily_ticket_distribution
USING DELTA
LOCATION '/mnt/gold/sales/daily_ticket_distribution';
    """
)

In [0]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.store_performance
USING DELTA
LOCATION '/mnt/gold/sales/store_performance';
    """
)